In [105]:
import json
import pickle
import logging
import numpy as np
import torch
import torch.utils.data
from torch.utils.data import Dataset
from itertools import chain
import tarfile
from argparse import ArgumentParser
from utils.meta.dataset import get_dataset, build_input_from_segments
from utils.meta.VideoGPT2 import *
from utils.meta.train import SPECIAL_TOKENS, SPECIAL_TOKENS_DICT


In [112]:
import pandas as pd
import json
from pandas import json_normalize

text_data = json.load(open('./data/annotations/test_set.json','r'))
text_data = json_normalize(text_data['dialogs']) 
text_data = text_data.explode('dialog').reset_index(drop=True)
text_data = text_data.join(pd.json_normalize(text_data.dialog)).drop(columns=['dialog'])
text_data.head()


,image_id,summary,caption,answer,question
0,YSE1G,the girl walks into a room with a dog with a t...,a person walked through a doorway into the liv...,there is only one person and a dog .,is there only one person ?
1,YSE1G,the girl walks into a room with a dog with a t...,a person walked through a doorway into the liv...,she walks in from outside with the towel aroun...,does she walk in with a towel around her neck ?
2,YSE1G,the girl walks into a room with a dog with a t...,a person walked through a doorway into the liv...,she does not interact with the dog,does she interact with the dog ?
3,YSE1G,the girl walks into a room with a dog with a t...,a person walked through a doorway into the liv...,she dropped the towel on the floor at the end ...,does she drop the towel on the floor ?
4,YSE1G,the girl walks into a room with a dog with a t...,a person walked through a doorway into the liv...,there was nothing else interacting but before ...,so she walks in to a room with a dog and drops...


In [107]:
text_data.shape

(7330, 5)

In [108]:
text_data['image_id'].value_counts()

TUD6M    10
6FJBD    10
08LOY    10
RQRRD    10
MPHFU    10
         ..
CSHTJ    10
W9BWJ    10
B0SV3    10
5D85P    10
81R72    10
Name: image_id, Length: 733, dtype: int64

In [109]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7330 entries, 0 to 7329
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  7330 non-null   object
 1   summary   7330 non-null   object
 2   caption   7330 non-null   object
 3   answer    7330 non-null   object
 4   question  7330 non-null   object
dtypes: object(5)
memory usage: 286.5+ KB


In [110]:
my_tar = tarfile.open('./data/annotations.tar')
my_tar.extractall('./data/annotations') # specify which folder to extract to
my_tar.close()

In [111]:
def get_npy_shape(filename):
    print('read npy file header and return its shape')
    with open(filename, 'rb') as f:
        if filename.endswith('.pkl'):
            shape = pickle.load(f).shape
        else:
            major, minor = np.lib.format.read_magic(f)
            shape, fortran, dtype = np.lib.format.read_array_header_1_0(f)
    return shape

get_npy_shape('/data/dataset/i3d_rgb_vgg19_4')

read npy file header and return its shape


FileNotFoundError: [Errno 2] No such file or directory: '/data/dataset/i3d_rgb_vgg19_4'

In [ ]:
filename = '/data/dataset/i3d_rgb_vgg19_4'
with open(filename, 'rb') as f:
    if filename.endswith('.pkl'):
        shape = pickle.load(f).shape
    else:
        major, minor = np.lib.format.read_magic(f)
        shape, fortran, dtype = np.lib.format.read_array_header_1_0(f)

text_data.